In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances

from sklearn.preprocessing import StandardScaler


In [2]:
relevent_threshold = 3.5

**Load the dataset**

In [3]:
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')
merged_df = pd.merge(ratings_df, movies_df, on='movieId')
# Filter out users who have rated fewer movies (optional)
user_counts = ratings_df['userId'].value_counts()
active_users = user_counts[user_counts >= 20].index
ratings_df = ratings_df[ratings_df['userId'].isin(active_users)]

ratings_df.head()
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


#Split the training set and test set

In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(ratings_df, test_size=0.2, random_state=42)

In [5]:
train_set.head()

,userId,movieId,rating,timestamp
37865,273,5816,4.5,1466946328
46342,339,2028,4.5,1446663181
64614,461,3895,0.5,1093224965
41974,300,3578,4.5,1086010878
50236,369,292,3.0,847465462


In [6]:
test_set.head()

,userId,movieId,rating,timestamp
19090,128,1028,5.0,1049690908
99678,665,4736,1.0,1010197684
18455,120,4002,3.0,1167420604
35755,257,1274,4.0,1348544094
66536,468,6440,4.0,1296191715


**Create the User-Item Matrix**

In [7]:
user_item_matrix = train_set.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)
user_item_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,160656,160718,161084,161155,161594,161830,161918,161944,162542,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0


In [8]:
# Calculate User Similarity
user_similarity = cosine_similarity(user_item_matrix)
np.fill_diagonal(user_similarity, 0)  # Set diagonal elements to 0 to avoid self-similarity
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

user_similarity_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.085679,0.019716,0.000000,0.084181,0.000000,0.014354,0.000000,...,0.000000,0.000000,0.003314,0.035803,0.000000,0.000000,0.000000,0.075423,0.000000,0.020265
2,0.000000,0.000000,0.092037,0.081137,0.072216,0.000000,0.149972,0.084527,0.110684,0.051020,...,0.374325,0.069295,0.075608,0.149802,0.462915,0.339362,0.042671,0.021810,0.112026,0.074662
3,0.000000,0.092037,0.000000,0.050344,0.065398,0.085793,0.061324,0.143879,0.104469,0.017533,...,0.052034,0.029244,0.129791,0.128113,0.121741,0.023173,0.064155,0.025504,0.065198,0.081216
4,0.085679,0.081137,0.050344,0.000000,0.103764,0.092846,0.251964,0.153572,0.035802,0.098074,...,0.086965,0.035391,0.109564,0.215650,0.102951,0.060254,0.085024,0.102528,0.050497,0.204885
5,0.019716,0.072216,0.065398,0.103764,0.000000,0.086933,0.060305,0.119289,0.080419,0.040818,...,0.177942,0.024758,0.111415,0.177003,0.130934,0.062006,0.054312,0.024675,0.053662,0.181957


#User-based Movie Recommender


**Rating Predictions Approach**

In [9]:
test_ratings = test_set.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)
train_ratings = train_set.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)

In [10]:
def calculate_rmse(predicted_ratings, test_ratings):
    squared_errors = []
    predicted_ratings.fillna(0)
    test_ratings.fillna(0)
    for user_id in test_ratings.index:
        if user_id in predicted_ratings.index:
          common_movies = predicted_ratings.loc[user_id][predicted_ratings.loc[user_id] > 0].index.intersection(test_ratings.loc[user_id][test_ratings.loc[user_id] > 0].index)

          squared_diff = (predicted_ratings.loc[user_id, common_movies] - test_ratings.loc[user_id, common_movies]) ** 2
          squared_errors.extend(squared_diff.tolist())

    mean_squared_error = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_error)

    return rmse


In [11]:
# Get the mean_ratings
mean_rating = ratings_df['rating'].mean()
mean_ratings = ratings_df.groupby('movieId')['rating'].mean()
num_ratings = ratings_df.groupby('movieId')['rating'].count()

mean_ratings[num_ratings <= 50] = mean_rating

print(mean_ratings)

def generate_user_based_predictions_test(user_id, top_n=5):
    # Find similar users in the user neighborhood
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:top_n+1].index

    # Get movies rated by the target user
    user_movies = user_item_matrix.loc[user_id]

    # Calculate the weighted average of ratings based on similar users' ratings
    similarity_sum = user_similarity_df.loc[user_id, similar_users].sum()
    predicted_ratings = np.dot(user_item_matrix.loc[similar_users].T, user_similarity_df.loc[user_id, similar_users])
    predicted_ratings /= similarity_sum


    predicted_ratings = pd.Series(predicted_ratings, index = user_movies.index)

    # Fill missing predicted ratings with the mean rating of the movie
    missing_ratings = predicted_ratings[predicted_ratings == 0]
    missing_item_ids = missing_ratings.index
    default_ratings = mean_ratings[missing_item_ids]
    predicted_ratings[missing_item_ids] = default_ratings

    # Exclude movies that the target user has already rated
    predicted_ratings = predicted_ratings[user_movies == 0]

    # Sort the predicted ratings in descending order
    predicted_ratings = predicted_ratings.sort_values(ascending=False)

    predicted_ratings = predicted_ratings[~predicted_ratings.index.duplicated()]


    return predicted_ratings


movieId
1         3.872470
2         3.401869
3         3.161017
4         3.543608
5         3.267857
            ...   
161944    3.543608
162376    3.543608
162542    3.543608
162672    3.543608
163949    3.543608
Name: rating, Length: 9066, dtype: float64


**User_id**

In [12]:
selected_id= input(' Enter the userId of the user to recommend to : ')
selected_id = int(selected_id)

 Enter the userId of the user to recommend to : 15


In [13]:
print("List of recommendations for user {}".format(selected_id))
pred_rating = generate_user_based_predictions_test(selected_id, top_n = 5)
pred_rating.sort_values(inplace=True,ascending=False)
pred_rating_df = pd.DataFrame(pred_rating).reset_index()
pred_rating_df.columns = ['movieId', 'predicted_rating']
final_pred_df = pd.merge(pred_rating_df,movies_df)
final_pred_df.head(10)

List of recommendations for user 15


,movieId,predicted_rating,title,genres
0,356,4.096833,Forrest Gump (1994),Comedy|Drama|Romance|War
1,1028,3.814103,Mary Poppins (1964),Children|Comedy|Fantasy|Musical
2,1225,3.796217,Amadeus (1984),Drama
3,261,3.715686,Little Women (1994),Drama
4,5952,3.697549,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
5,661,3.632075,James and the Giant Peach (1996),Adventure|Animation|Children|Fantasy|Musical
6,1517,3.593776,Austin Powers: International Man of Mystery (1...,Action|Adventure|Comedy
7,596,3.583333,Pinocchio (1940),Animation|Children|Fantasy|Musical
8,1022,3.563636,Cinderella (1950),Animation|Children|Fantasy|Musical|Romance
9,2668,3.543608,Swamp Thing (1982),Horror|Sci-Fi
